In [1]:
import pandas as pd
import numpy as np

# Hierarchical Indexing

In [2]:
data = pd.Series(np.random.randn(9),
index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],[1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.436153
   2   -1.575676
   3   -0.336152
b  1   -2.057215
   3    0.493259
c  1   -0.030016
   2    1.596835
d  2    0.564247
   3   -0.687131
dtype: float64

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [5]:
# data['a']
# data['b']
# data['c']
# data['b':'d']
# data.loc[['a','c']]
data.loc[:,2]

a   -1.575676
c    1.596835
d    0.564247
dtype: float64

In [6]:
# data.unstack(0)
data.unstack(1)
# data.unstack().stack()

,1,2,3
a,-0.436153,-1.575676,-0.336152
b,-2.057215,NaN,0.493259
c,-0.030016,1.596835,NaN
d,NaN,0.564247,-0.687131


In [7]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['b', 'b', 'a', 'a'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],['Green', 'Red', 'Green']])

# frame.unstack()
# frame.stack()
frame

Ohio     Colorado
    Green Red    Green
b 1     0   1        2
  2     3   4        5
a 1     6   7        8
  2     9  10       11

In [8]:
# frame.index
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
b    1        0   1        2
     2        3   4        5
a    1        6   7        8
     2        9  10       11

In [9]:
# multiindexing can be created from the method below
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reordering and Sorting Levels

In [10]:
frame.swaplevel('key1', 'key2')  # Reodering the dataset by index

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    b        0   1        2
2    b        3   4        5
1    a        6   7        8
2    a        9  10       11

In [11]:
print(frame.sort_index(level=0))
print()
print(frame.sort_index(level=1))    # DF.sort_values() can be used for sorting values

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        6   7        8
     2        9  10       11
b    1        0   1        2
     2        3   4        5

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        6   7        8
b    1        0   1        2
a    2        9  10       11
b    2        3   4        5


In [12]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        6   7        8
     b        0   1        2
2    a        9  10       11
     b        3   4        5

### Summary Statistics by Level

In [13]:
# frame.sum(level='key1').sort_index(level=0)
frame.sum(level='color', axis= 1)

color      Green  Red
key1 key2            
b    1         2    1
     2         8    4
a    1        14    7
     2        20   10

### Indexing with a DataFrame’s columns

In [14]:
frame2 = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two','two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})

frame2

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [15]:
# we can prevet set_index method from droping indexed cols
frame2_prt = frame2.set_index(['c', 'd'], drop=False)
frame2_prt

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [16]:
# frame2 = frame.set_index(['c', 'd'])
frame2.reset_index()

,index,a,b,c,d
0,0,0,7,one,0
1,1,1,6,one,1
2,2,2,5,one,2
3,3,3,4,two,0
4,4,4,3,two,1
5,5,5,2,two,2
6,6,6,1,two,3


# Combining and Merging Datasets

### Database-Style DataFrame Joins

#### many-to-one

In [17]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],'data2': range(3)})

print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6
  key  data2
0   a      0
1   b      1
2   d      2


In [18]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [19]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


In [20]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],'data1': range(7)})

df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],'data2': range(3)})

print(df3)
print(df4)

  lkey  data1
0    b      0
1    b      1
2    a      2
3    c      3
4    a      4
5    a      5
6    b      6
  rkey  data2
0    a      0
1    b      1
2    d      2


In [21]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')  # if find different keys then specify them

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


In [22]:
print(pd.merge(df1, df2, how='inner'))   # keys which are in both sets
print(pd.merge(df1, df2, how='outer'))   # keys all from boht sets
print(pd.merge(df1, df2, how='left'))    # keys all from left and matched from right
print(pd.merge(df1, df2, how='right'))   # keys all from right and matched from left


  key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0
  key  data1  data2
0   b    0.0    1.0
1   b    1.0    1.0
2   b    6.0    1.0
3   a    2.0    0.0
4   a    4.0    0.0
5   a    5.0    0.0
6   c    3.0    NaN
7   d    NaN    2.0
  key  data1  data2
0   b      0    1.0
1   b      1    1.0
2   a      2    0.0
3   c      3    NaN
4   a      4    0.0
5   a      5    0.0
6   b      6    1.0
  key  data1  data2
0   b    0.0      1
1   b    1.0      1
2   b    6.0      1
3   a    2.0      0
4   a    4.0      0
5   a    5.0      0
6   d    NaN      2


#### many-to-many

In [23]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],'data2': range(5)})
print(df1)
print(df2)

  key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   b      5
  key  data2
0   a      0
1   b      1
2   a      2
3   b      3
4   d      4


In [24]:
# pd.merge(df1, df2, on='key', how='left')
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


In [25]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
....: 'key2': ['one', 'two', 'one'],
....: 'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
....: 'key2': ['one', 'one', 'one', 'two'],
....: 'rval': [4, 5, 6, 7]})


In [26]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [27]:
# pd.merge(left, right, on='key1')
pd.merge(left, right, on='key1', how='outer', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [28]:
# pd.merge(left, right, on='key1',how='inner', indicator=True)
pd.merge(left, right,left_index=True, right_index=True)

,key1_x,key2_x,lval,key1_y,key2_y,rval
0,foo,one,1,foo,one,4
1,foo,two,2,foo,one,5
2,bar,one,3,bar,one,6


In [29]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
....: 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

print(left1)
print()
print(right1)

  key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

   group_val
a        3.5
b        7.0


In [30]:
# pd.merge(left1, right1, left_on='key', right_index=True)
pd.merge(left1, right1, left_on='key', right_index=True, how = 'outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


In [31]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
....: 'Nevada', 'Nevada'],
....: 'key2': [2000, 2001, 2002, 2001, 2002],
....: 'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
....: index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
....: 'Ohio', 'Ohio'],
....: [2001, 2000, 2000, 2000, 2001, 2002]],
....: columns=['event1', 'event2'])

print(lefth)
righth

     key1  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0


event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [32]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [33]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)
#handling the duplicate index wiht how='outer'
pd.merge(lefth, righth,how='outer',  left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


In [34]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
....: index=['a', 'c', 'e'],
....: columns=['Ohio', 'Nevada'])

right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
....: index=['b', 'c', 'd', 'e'],
....: columns=['Missouri', 'Alabama'])

print(left2)
right2

   Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0


,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [35]:
print(pd.merge(left2, right2, how='outer', left_index=True, right_index=True))
print()
print(left2.join(right2, how='outer'))  # for conveniece we can use join()

   Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

   Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0


##### joining

In [36]:
left1.join(right1, on='key')    # also supports joining of passed index

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


In [37]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
....: index=['a', 'c', 'e', 'f'],
....: columns=['New York', 'Oregon'])

another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [38]:
# left2.join([right2, another])  # we can also pass a list of dataframes to the join()
left2.join([right2, another], how= 'outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenating Along an Axis

In [39]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [40]:
np.concatenate([arr, arr], axis = 1)
# np.concatenate([arr, arr], axis = 0)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [41]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])
print(s1,s2,s3, sep = '\n' )

a    0
b    1
dtype: int64
c    2
d    3
e    4
dtype: int64
f    5
g    6
dtype: int64


In [42]:
pd.concat([s1, s2, s3] )
# pd.concat([s1, s2, s3], axis=0)
pd.concat([s1, s2, s3], axis = 1)

C:\Users\zubim\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [43]:
s4 = pd.concat([s1, s3])

In [44]:
pd.concat([s1, s2, s3], axis = 1, join = 'outer')

C:\Users\zubim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [45]:
pd.concat([s1, s4], axis=1, join_axes = [['a', 'c', 'b', 'e']])


C:\Users\zubim\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The join_axes-keyword is deprecated. Use .reindex or .reindex_like on the result to achieve the same functionality.
  """Entry point for launching an IPython kernel.


,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,1.0
e,NaN,NaN


In [46]:
# pd.concat([s1, s2, s3], axis = 1, keys=['a', 'b', 'c'], sort = False)
result = pd.concat([s1, s2, s3], keys=['a', 'b', 'c'], sort = False)
result.unstack()

,a,b,c,d,e,f,g
a,0.0,1.0,NaN,NaN,NaN,NaN,NaN
b,NaN,NaN,2.0,3.0,4.0,NaN,NaN
c,NaN,NaN,NaN,NaN,NaN,5.0,6.0


In [47]:
pd.concat([s1, s2, s3], axis = 0, keys=['a', 'b', 'c'], sort = False)

a  a    0
   b    1
b  c    2
   d    3
   e    4
c  f    5
   g    6
dtype: int64

In [48]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],columns=['three', 'four'])
print(df1)

print(df2)

   one  two
a    0    1
b    2    3
c    4    5
   three  four
a      5     6
c      7     8


In [49]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'], sort='False')

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [50]:
pd.concat({'level1': df1, 'level2': df2}, axis=1, sort="False")

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

In [51]:
pd.concat([df1, df2], ignore_index=True, sort= True)

,four,one,three,two
0,NaN,0.0,NaN,1.0
1,NaN,2.0,NaN,3.0
2,NaN,4.0,NaN,5.0
3,6.0,NaN,5.0,NaN
4,8.0,NaN,7.0,NaN


In [52]:
pd.concat([s1, s2], sort= False, verify_integrity=False, join='outer')

a    0
b    1
c    2
d    3
e    4
dtype: int64

### Combining Data with Overlap

In [53]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
.....: index=['f', 'e', 'd', 'c', 'b', 'a'])

b = pd.Series(np.arange(len(a), dtype=np.float64),
.....: index=['f', 'e', 'd', 'c', 'b', 'a'])
print(a)
print()
print(b)

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64


In [54]:
b[-1] = np.nan

b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [55]:
np.where(pd.isnull(a), b, a)   # if a have null then use b's value there otherwise use a's value

array([0. , 2.5, 2. , 3.5, 4.5, nan])

In [56]:
print(b[:-2])
a[2:]

f    0.0
e    1.0
d    2.0
c    3.0
dtype: float64


d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [57]:
b[:-2].combine_first(a[2:])     #  np.where(pd.isnull(a), b, a)   equivelent to this
a[2:].combine_first(b[:-2])

a    NaN
b    4.5
c    3.5
d    2.0
e    1.0
f    0.0
dtype: float64

In [58]:
df3 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
.....: 'b': [np.nan, 2., np.nan, 6.],
.....: 'c': range(2, 18, 4)})

df4 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
.....: 'b': [np.nan, 3., 4., 6., 8.]})

print(df3, end= '\n\n')
print(df4)

     a    b   c
0  1.0  NaN   2
1  NaN  2.0   6
2  5.0  NaN  10
3  NaN  6.0  14

     a    b
0  5.0  NaN
1  4.0  3.0
2  NaN  4.0
3  3.0  6.0
4  7.0  8.0


In [59]:
df3.combine_first(df4)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


# Reshaping and Pivoting

### stack  and  unstack

In [60]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
.....: index=pd.Index(['Ohio', 'Colorado'], name='state'),
.....: columns=pd.Index(['one', 'two', 'three'],
.....: name='number'))

data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [61]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

In [62]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [63]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [64]:
# result.unstack('number')
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [65]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

print(s1, s2, sep = '\n\n')


a    0
b    1
c    2
d    3
dtype: int64

c    4
d    5
e    6
dtype: int64


In [66]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [67]:
data2.unstack(1).stack(dropna = False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

In [68]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [71]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [88]:
df.unstack('state', 1)

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

In [90]:
# df.unstack('number').stack('side')
df.unstack('state').stack('state')

side             left  right
number state                
one    Ohio         0      5
       Colorado     3      8
two    Ohio         1      6
       Colorado     4      9
three  Ohio         2      7
       Colorado     5     10

### Pivoting “Long” to “Wide” Format

In [125]:
data = pd.read_csv('./Data/macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [126]:
periods = pd.PeriodIndex(year = data.year, quarter = data.quarter, name = 'date')

columns = pd.Index(['realgdp', 'infl', 'unemp'], name = 'item')

data = data.reindex(columns=columns)

data.index = periods.to_timestamp('D', 'end')

ldata = data.stack().reset_index().rename(columns = {0:'value'})

ldata[:10]

In [135]:
pivot_table = ldata.pivot('date', 'item', 'value')
pivot_table

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


In [137]:
ldata['value2'] = np.random.randn(len(ldata))
ldata.head(10)

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,1.512288
1,1959-03-31 23:59:59.999999999,infl,0.000,-0.967630
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.181602
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.447915
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.847437
5,1959-06-30 23:59:59.999999999,unemp,5.100,1.148622
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,2.438322
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.562238
8,1959-09-30 23:59:59.999999999,unemp,5.300,1.540315
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,1.531140


In [148]:
pivot_t2 = ldata.pivot('date', 'item')
pivot_t2.head(10)
# pivot_t2['value'][:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8 -0.967630  1.512288   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.847437  1.447915   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.562238  2.438322   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6 -0.241661  1.531140   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -1.416644  1.013734   
1960-06-30 23:59:59.999999999  0.14  2834.390   5.2 -1.223799  2.385906   
1960-09-30 23:59:59.999999999  2.70  2839.022   5.6 -0.288093 -0.209299   
1960-12-31 23:59:59.999999999  1.21  2802.616   6.3  0.751436 -0.043684   
1961-03-31 23:59:59.999999999 -0.40  2819.264   6.8  0.818629 -2.123106   
1961-06-30 23:59:59.999999999  1.47  2872.005   7.0  0.214944  1.335564   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.181602  
1959-06-30 23:59:59.999999999  1.148622  
1959-09-30 23:59:59.999999999  1.540315  
1959-12-31 23:59:59.999999999  0.919614  
1960-03-31 23:59:59.999999999  0.566411  
1960-06-30 23:59:59.999999999  0.772708  
1960-09-30 23:59:59.999999999  0.102003  
1960-12-31 23:59:59.999999999 -0.760618  
1961-03-31 23:59:59.999999999  0.768733  
1961-06-30 23:59:59.999999999  1.109469

In [150]:
pivot_method = ldata.set_index(['date', 'item']).unstack('item')
pivot_method

value                     value2            \
item                           infl    realgdp unemp      infl   realgdp   
date                                                                       
1959-03-31 23:59:59.999999999  0.00   2710.349   5.8 -0.967630  1.512288   
1959-06-30 23:59:59.999999999  2.34   2778.801   5.1 -0.847437  1.447915   
1959-09-30 23:59:59.999999999  2.74   2775.488   5.3 -0.562238  2.438322   
1959-12-31 23:59:59.999999999  0.27   2785.204   5.6 -0.241661  1.531140   
1960-03-31 23:59:59.999999999  2.31   2847.699   5.2 -1.416644  1.013734   
...                             ...        ...   ...       ...       ...   
2008-09-30 23:59:59.999999999 -3.16  13324.600   6.0  1.541587  0.353029   
2008-12-31 23:59:59.999999999 -8.79  13141.920   6.9 -0.063434  0.344074   
2009-03-31 23:59:59.999999999  0.94  12925.410   8.1 -0.014434  1.058691   
2009-06-30 23:59:59.999999999  3.37  12901.504   9.2  0.752903 -0.090408   
2009-09-30 23:59:59.999999999  3.56  12990.341   9.6  0.160315  0.306127   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  0.181602  
1959-06-30 23:59:59.999999999  1.148622  
1959-09-30 23:59:59.999999999  1.540315  
1959-12-31 23:59:59.999999999  0.919614  
1960-03-31 23:59:59.999999999  0.566411  
...                                 ...  
2008-09-30 23:59:59.999999999 -0.180135  
2008-12-31 23:59:59.999999999  1.626343  
2009-03-31 23:59:59.999999999  1.141059  
2009-06-30 23:59:59.999999999 -0.331415  
2009-09-30 23:59:59.999999999 -0.480684  

[203 rows x 6 columns]

### Pivoting “Wide” to “Long” Format

In [157]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'], 'A': [1, 2, 3], 'B': [4, 5, 6], 'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


In [161]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


In [165]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


In [166]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


In [167]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
